In [ ]:
from numpy import zeros, array

#return True if the list of integers contains a integer and its opposite
# ex.is_tautologie([1,2,3,-1,5]) return True
def is_tautologie(clause:list)->bool:
    for i in clause:
        op_i = -i
        if op_i in clause: return True
    return False

def regle_1(clauses:list)->list:
    """Règle 1 : oter tautologie -> clause contenant l et non l"""
    l2 = []
    for c in clauses:
        if not is_tautologie(c): l2.append(c)
    if verbose:
        if not clauses == l2:
            print("règle 1 activée")
    return l2

l1 = [[1,2],[1,4,-1], [8,1,6,4,9,1,-6], [-2, 4], [1]]

In [ ]:
################################################

def ote_val_from_clauses(clauses:list, value:int)->list:
    liste2 = []
    for c in clauses:
        c2 = [i for i in c if i != value]
        liste2.append(c2)
    return liste2

def ote_clauses_with_val(clauses:list, value:int)->list:
    liste2 = []
    for c in clauses:
        if not value in c:
            liste2.append(c)
    return liste2


def regle_2(clauses:list)->list:
    """Règle 2 : clause contient 1 seul littéral -> enlever les clauses le contenant, et enlever l'apparition de son inverse ailleurs"""
    for c in clauses:
        if len(c) == 1:
            value = c[0]
            clauses2 = ote_clauses_with_val(clauses, value)
            clauses3 = ote_val_from_clauses(clauses2, -value)
            if verbose: print("regle 2 activee, retrait de ", value)
            return clauses3
    return clauses

l2 = [[1],[1,2], [1,6,4], [-2, 4], [-1,7]]

In [ ]:
################################################

def exist_in_clauses(value:int, clauses:list)->bool:
    """retourne vrai si la valeur value existe dans au moins une clause"""
    for c in clauses:
        if value in c: return True
    return False

def single_lit(clauses:list)->int:
    """retourne un literal qui apparait dans des clauses mais dont l'opposé n'apparait jamais"""
    for c in clauses:
        for i in c:
            if not exist_in_clauses(-i, clauses): return i
    return 0


def regle_3(clauses:list)->list:
    """Règle 3 : 1 littéral apparaît dans des clauses, son inverse n'apparait jamais -> enlever les clauses le contenant"""
    lit = single_lit(clauses)
    if lit != 0:
        clauses2 = ote_clauses_with_val(clauses, lit)
        if verbose: print("regle 3 activee, retrait de ", lit)
        return clauses2
    return clauses

l3 = [[1,2], [1,6,4], [-2, 4]]

In [ ]:
################################################

def bigger_clauses(clauses:list)->list:
    """retourne les clauses qui contiennent d'autres clauses"""
    clauses2 = []
    for c in clauses:
        for c2 in clauses:
            if c != c2:
                if  all(elem in c for elem in c2):
                    clauses2.append(c)
    return clauses2


def regle_4(clauses:list)->list:
    """Règle 4 : si une  clause est contenu dans d'autres -> enlever les autres"""
    clauses2 = bigger_clauses(clauses)
    clauses3 = [c for c in clauses if c not in clauses2]
    if verbose:
        if clauses2!=[]:print("regle 4 activee, retrait des clause qui contiennent ", clauses2)
    return clauses3


l4 = [[1,2], [1,6,4], [1,2, 4], [1,6]]

In [ ]:
################################################

def get_not_single(clauses:list)->int:
    """retourne un littéral l dont son inverse apparaît également """
    for c in clauses:
        for i in c:
            if  exist_in_clauses(-i, clauses): return i


def regle_5(clauses:list)->list:
    """Règle 5 : Créer des mondes -> choisir un littéral l dont son inverse apparaît également
    -> créer 2 formules,
    - F1) contenant les clauses de F sauf celles contenant l
         et où les apparitions de ¬l sont ôtées
    - F2) contenant les clauses de F sauf celles contenant ¬l
      et où les apparitions de l sont ôtées
    """
    l = get_not_single(clauses)
    if l != 0:
        clauses21 = ote_clauses_with_val(clauses, l)
        clauses31 = ote_val_from_clauses(clauses21, -l)
        clauses22 = ote_clauses_with_val(clauses, -l)
        clauses32 = ote_val_from_clauses(clauses22, l)
        if verbose: print("regle 5 activee, creation de deux mondes à partir de ", l," et de ", -l )
        return (clauses31, clauses32)

l5 = [[1,2], [1,6,4], [-1,2, 4], [5,6], [4,5,-1]]

In [ ]:
################################################

def formules_egales(f1:list, f2:list)->bool:
    """retourne vrai si les formules f1 et f2 sont identiques"""
    ##TODO: utiliser des sorted set pour une "vraie" comparaison
    if len(f1) != len(f2): return False
    for c in f1:
        if c not in f2: return False
    return True

cpt = 0
verbose = False
def DP(clauses:list)->bool:
    """retourne vrai si le DPLL est fini"""
    global cpt
    cpt +=1
    print("résolution de ", clauses)
    clr1 = regle_1(clauses)
    if len(clr1) == 0:
        print("succes")
        return True
    if [] in clr1:
        print("echec")
        return False
    clr2 = regle_2(clr1)
    if len(clr2) == 0:
        print("succes")
        return True
    if [] in clr2:
        print("echec")
        return False
    if not formules_egales(clr1, clr2): return DP(clr2)
    clr3 = regle_3(clr2)
    if len(clr3) == 0:
        print("succes")
        return True
    if [] in clr3:
        print("echec")
        return False
    if not formules_egales(clr2, clr3): return DP(clr3)
    clr4 = regle_4(clr3)
    if len(clr4) == 0:
        print("succes")
        return True
    if [] in clr4:
        print("echec")
        return False
    if not formules_egales(clr3, clr4): return DP(clr4)
    clr51,clr52 = regle_5(clr4)
    mondes_unis = DP(clr51) or DP(clr52)
    print("fin résolution de ", clauses)
    return mondes_unis


def lire_cnf(fichier):
    """
    Lit un fichier CNF en format DIMACS et retourne une liste de listes représentant les clauses.

    :param fichier: Le chemin du fichier CNF à lire au format DIMACS comme :
    p cnf 3 4
    -1 2 0
    -2 3 0
    1 0
    -3 0
    :return: Une liste de listes où chaque sous-liste représente une clause. comme :
    [[-1,2], [-2,3], [1], [3]]
    """
    clauses = []

    with open(fichier, 'r') as f:
        for ligne in f:
            if ligne.startswith("%"):break
            # Ignorer les lignes de commentaires et le préambule
            if ligne.startswith('c') or ligne.startswith('p'):
                continue

            # Diviser la ligne en entiers
            literals = list(map(int, ligne.split()))

            # Retirer le 0 de fin de clause
            if literals[-1] == 0:
                literals.pop()

            # Ajouter la clause à la liste des clauses
            if literals:
                clauses.append(literals)

    return clauses


exo4 = [[1, -2, 3, -4, -5, -6], [2, 3], [2, 3, 4, 7, 6], [2, -4, 7, -5], [2, -3, 8, -6], [2, -4, 5, -8], [-2, -3], [-2, -3, -4], [-2, 3, 4], [-2, -4, 7], [-2, -7, 5], [-8, -6], [8, 6], [-8, 6]]

# p = 1
# q = 2
# r = 3
# s = 4
# t = 5
# exo2 = [[p, q], [-p, r], [-q, r, s], [-r], [-s], [-t] ]
exo2 = [[1, 2], [-1, 3], [-2, 3, 4], [-3], [-4], [-5] ]

a = 1
b = 2
c = 3
d = 4
p = 5
exo3 = [[a,b], [-a, c], [-b, d], [-c, p], [-d, p], [-p, -c]]

# ***A rendre***

# ***DP v2***

In [ ]:
from collections import Counter

def regle_2_optimisee(clauses):
    """
    Règle 2 : Propagation unitaire.
    Identifie les littéraux unitaires (clauses de taille 1) et :
    - Supprime toutes les clauses satisfaites par ces littéraux.
    - Supprime également les négations de ces littéraux dans les autres clauses.

    Optimisation :
    - Réduction immédiate des clauses pour limiter la taille du problème.
    - Itérations jusqu'à ce qu'il n'y ait plus de littéraux unitaires.
    """
    while True:
        unit_clauses = [c[0] for c in clauses if len(c) == 1]
        if not unit_clauses:
            break
        for lit in unit_clauses:
            clauses = [c for c in clauses if lit not in c]
            clauses = [[l for l in c if l != -lit] for c in clauses]
    return clauses

def regle_5_optimisee(clauses):
    """
    Règle 5 : Heuristique de choix de littéral.
    Sélectionne le littéral le plus fréquent dans les clauses pour maximiser l'impact de la division.
    Divise les clauses en deux ensembles :
    - `clauses1` pour l'hypothèse que le littéral est vrai.
    - `clauses2` pour l'hypothèse que le littéral est faux.

    Optimisation :
    - Analyse des fréquences des littéraux pour choisir celui qui maximise la réduction des clauses.
    - Division simplifiée en deux branches pour limiter l'exploration de l'espace de recherche.
    """
    literal_counts = Counter(lit for c in clauses for lit in c)
    chosen_lit = max(literal_counts, key=literal_counts.get)
    clauses1 = [c for c in clauses if chosen_lit not in c]
    clauses1 = [[l for l in c if l != -chosen_lit] for c in clauses1]
    clauses2 = [c for c in clauses if -chosen_lit not in c]
    clauses2 = [[l for l in c if l != chosen_lit] for c in clauses2]
    return clauses1, clauses2

def DP_v2(clauses):
    """Davis-Putnam optimisé (version 2).
    Applique successivement des règles de simplification avant de diviser récursivement les clauses.
    Utilise des règles améliorées (propagation unitaire, choix heuristique, etc.).

    Optimisations :
    - Applique les règles de simplification dans un ordre logique pour réduire au maximum les clauses avant la récursion.
    - Utilise des heuristiques pour choisir efficacement les littéraux dans la division."""
    global cpt
    cpt += 1
    clauses = regle_1(clauses)  # Supprimer les tautologies
    if not clauses:
        return True
    if [] in clauses:
        return False
    clauses = regle_2_optimisee(clauses)  # Propagation unitaire
    if not clauses:
        return True
    if [] in clauses:
        return False
    clauses = regle_3(clauses)  # Littéraux isolés
    if not clauses:
        return True
    if [] in clauses:
        return False
    clauses = regle_4(clauses)  # Clauses redondantes
    if not clauses:
        return True
    if [] in clauses:
        return False
    clauses1, clauses2 = regle_5_optimisee(clauses)  # Division heuristique
    return DP_optimise(clauses1) or DP_optimise(clauses2)


# ***DP optimise***

In [ ]:
from collections import Counter

def propagation_unitaire_DP(clauses):
    """
    Propagation unitaire optimisée.
    Simplifie les clauses en résolvant les littéraux unitaires (clauses de taille 1) :
    - Si un littéral est trouvé, on supprime toutes les clauses où il apparaît.
    - On supprime également sa négation dans les autres clauses.
    - Si une clause devient vide, la formule est insatisfiable.

    Optimisations :
    - Utilisation d'une file dynamique (`unit_queue`) pour traiter efficacement les littéraux unitaires.
    - Réduction immédiate des clauses pour limiter la taille du problème.
    """
    unit_queue = [c[0] for c in clauses if len(c) == 1]
    while unit_queue:
        lit = unit_queue.pop()
        new_clauses = []
        for clause in clauses:
            if lit in clause:
                continue  # Clause satisfaite
            if -lit in clause:
                new_clause = [x for x in clause if x != -lit]
                if not new_clause:  # Clause vide, insatisfiable
                    return False, []
                if len(new_clause) == 1:
                    unit_queue.append(new_clause[0])  # Ajouter à la file
                new_clauses.append(new_clause)
            else:
                new_clauses.append(clause)
        clauses = new_clauses
    return True, clauses


def choix_litteral(clauses):
    """
    Heuristique de choix de littéral.
    Sélectionne le littéral ayant la fréquence maximale dans les clauses.

    Optimisation :
    - Utilisation des fréquences des littéraux pour maximiser l'impact de la division.
    - Sélection d'un littéral susceptible de réduire rapidement la taille des clauses.
    """
    literal_counts = Counter(lit for clause in clauses for lit in clause)
    return max(literal_counts, key=literal_counts.get)

def division_optimisee(clauses, lit):
    """
       Division optimisée des clauses.
    Crée deux ensembles de clauses :
    - `pos_branch` : hypothèse que le littéral est vrai.
    - `neg_branch` : hypothèse que le littéral est faux.

    Optimisations :
    - Réduction directe des clauses en fonction de l'hypothèse sur le littéral.
    - Simplification immédiate des branches pour limiter la taille des sous-problèmes.
    """
    pos_branch = [c for c in clauses if lit not in c]
    pos_branch = [[l for l in c if l != -lit] for c in pos_branch]
    neg_branch = [c for c in clauses if -lit not in c]
    neg_branch = [[l for l in c if l != lit] for c in neg_branch]
    return pos_branch, neg_branch

def DP_optimise(clauses):
    """
    Algorithme Davis-Putnam optimisé.
    Applique une résolution récursive en combinant :
    - Une simplification par propagation unitaire.
    - Un choix heuristique de littéral.
    - Une division en deux branches (littéral positif et négatif).

    Optimisations :
    - La propagation unitaire réduit efficacement la taille des clauses avant la récursion.
    - Le choix heuristique des littéraux maximise l'efficacité de la division.
    - La division en branches simplifie l'exploration de l'espace de recherche.
    """
    global cpt
    cpt += 1

    # Propagation unitaire (version DP)
    satisfiable, clauses = propagation_unitaire_DP(clauses)
    if not satisfiable:
        return False
    if not clauses:  # Aucune clause restante, satisfiable
        return True

    # Choix heuristique du littéral
    lit = choix_litteral(clauses)

    # Division en deux branches
    pos_branch = [c for c in clauses if lit not in c]
    pos_branch = [[l for l in c if l != -lit] for c in pos_branch]
    neg_branch = [c for c in clauses if -lit not in c]
    neg_branch = [[l for l in c if l != lit] for c in neg_branch]

    # Résolution récursive
    return DP_optimise(pos_branch) or DP_optimise(neg_branch)



In [ ]:
import time
import pandas as pd

# Fonction pour lire un fichier DIMACS
def lire_cnf(fichier):
    clauses = []
    with open(fichier, 'r') as f:
        for ligne in f:
            ligne = ligne.strip()
            if not ligne or ligne.startswith(('c', 'p', '%', '0')):
                continue
            try:
                literals = list(map(int, ligne.split()))
                if literals and literals[-1] == 0:
                    literals.pop()
                if literals:
                    clauses.append(literals)
            except ValueError:
                continue
    return clauses

# Fonction pour tester DP Optimisé
def tester_DP_optimise(clauses):
    global cpt
    cpt = 0
    start_time = time.time()
    resultat = DP_optimise(clauses)
    elapsed_time = time.time() - start_time
    return resultat, elapsed_time, cpt

# Liste des fichiers pour les tests
fichiers = [
    "/content/drive/MyDrive/uf50/uf50-01.cnf",
    "/content/drive/MyDrive/uuf50/uuf50-04.cnf",
    "/content/drive/MyDrive/uuf150-01.cnf"
]
# Exécuter les tests
resultats = []
for fichier in fichiers:
    clauses = lire_cnf(fichier)
    dp_result, dp_time, dp_calls = tester_DP_optimise(clauses)
    resultats.append({
        "Fichier": fichier,
        "DP Résultat": dp_result,
        "DP Temps (s)": dp_time,
        "DP Appels": dp_calls
    })

# Afficher les résultats sous forme de tableau
df_resultats = pd.DataFrame(resultats)
print(df_resultats)


                                     Fichier  DP Résultat  DP Temps (s)  \
0    /content/drive/MyDrive/uf50/uf50-01.cnf         True      0.013709   
1  /content/drive/MyDrive/uuf50/uuf50-04.cnf        False      0.049182   
2       /content/drive/MyDrive/uuf150-01.cnf        False    140.805260   

   DP Appels  
0         48  
1        167  
2      84635  


# ***DPLL (pseudo-code)***

In [ ]:
from collections import Counter

def DPLL(clauses, assignment=None):
    """
    Implémentation optimisée de l'algorithme DPLL.
    :param clauses: Liste de clauses représentant la formule logique en CNF.
    :param assignment: Liste de littéraux déjà assignés (par défaut vide).
    :return: True si satisfiable, False sinon.
    """
    if assignment is None:
        assignment = []

    # Cas de base : toutes les clauses sont satisfaites
    if not clauses:
        return True

    # Cas de base : une clause vide existe -> insatisfiable
    if any(len(c) == 0 for c in clauses):
        return False

    # Simplification : propagation des littéraux unitaires
    unit_clauses = [c[0] for c in clauses if len(c) == 1]
    while unit_clauses:
        for lit in unit_clauses:
            assignment.append(lit)
            clauses = [c for c in clauses if lit not in c]  # Supprimer les clauses satisfaites
            clauses = [[l for l in c if l != -lit] for c in clauses]  # Supprimer les opposés
        unit_clauses = [c[0] for c in clauses if len(c) == 1]

    # Heuristique : choisir le littéral avec la fréquence maximale
    literal_counts = Counter(lit for c in clauses for lit in c)
    if not literal_counts:
        return True  # Toutes les clauses sont satisfaites
    chosen_lit = max(literal_counts, key=literal_counts.get)

    # Brancher sur le littéral choisi
    return (
        DPLL([c for c in clauses if chosen_lit not in c] +
             [[l for l in c if l != -chosen_lit] for c in clauses], assignment + [chosen_lit]) or
        DPLL([c for c in clauses if -chosen_lit not in c] +
             [[l for l in c if l != chosen_lit] for c in clauses], assignment + [-chosen_lit])
    )


# ***DPLL optimise***

In [ ]:
from collections import Counter

def propagation_unitaire_DPLL(clauses, assignment):
    """Propagation unitaire optimisée pour DPLL.
    Simplifie les clauses en résolvant les littéraux unitaires (clauses de taille 1) :
    - Si un littéral est trouvé, on supprime toutes les clauses où il apparaît.
    - On supprime également sa négation dans les autres clauses.
    - Si une clause devient vide, la formule est insatisfiable.
    Optimisation :
    - Utilisation d'une file dynamique (`unit_queue`) pour traiter les littéraux unitaires de manière efficace.
    - Réduction immédiate des clauses pour limiter la taille du problème à chaque étape.
    """
    unit_queue = [c[0] for c in clauses if len(c) == 1]
    while unit_queue:
        lit = unit_queue.pop()
        assignment.append(lit)
        new_clauses = []
        for clause in clauses:
            if lit in clause:
                continue  # Clause satisfaite
            if -lit in clause:
                new_clause = [x for x in clause if x != -lit]
                if not new_clause:  # Clause vide
                    return False, []
                if len(new_clause) == 1:
                    unit_queue.append(new_clause[0])  # Ajouter à la file
                new_clauses.append(new_clause)
            else:
                new_clauses.append(clause)
        clauses = new_clauses
    return True, clauses


def choix_litteral(clauses):
    """Choix heuristique du littéral basé sur les occurrences.
    Sélectionne le littéral ayant la fréquence maximale dans les clauses.

    Optimisation :
    - Analyse des fréquences des littéraux dans toutes les clauses.
    - Sélection du littéral avec le plus grand impact potentiel pour réduire rapidement la taille des clauses."""
    literal_counts = Counter(lit for clause in clauses for lit in clause)
    return max(literal_counts, key=literal_counts.get)

def DPLL_optimise(clauses):
    """Algorithme DPLL optimisé avec propagation unitaire et heuristiques.
    Fonction récursive qui applique :
    - Une simplification par propagation unitaire.
    - Un choix heuristique de littéral.
    - Une division récursive en deux branches (littéral positif et négatif).

    Optimisations :
    - Propagation unitaire rapide pour réduire les clauses avant la récursion.
    - Choix de littéraux basé sur leur fréquence pour maximiser la réduction des clauses.
    - Division en deux branches simplifiée pour explorer efficacement l'espace de recherche."""
    global cpt_dpll
    cpt_dpll += 1

    assignment = []
    satisfiable, clauses = propagation_unitaire_DPLL(clauses, assignment)
    if not satisfiable:
        return False
    if not clauses:  # Aucune clause restante, satisfiable
        return True

    lit = choix_litteral(clauses)
    # Branche positive
    satisfiable = DPLL_optimise([[l for l in c if l != -lit] for c in clauses if lit not in c])
    if satisfiable:
        return True
    # Branche négative
    return DPLL_optimise([[l for l in c if l != lit] for c in clauses if -lit not in c])



In [ ]:
import time
import pandas as pd

# Fonction pour lire un fichier DIMACS
def lire_cnf(fichier):
    clauses = []
    with open(fichier, 'r') as f:
        for ligne in f:
            ligne = ligne.strip()
            if not ligne or ligne.startswith(('c', 'p', '%', '0')):
                continue
            try:
                literals = list(map(int, ligne.split()))
                if literals and literals[-1] == 0:
                    literals.pop()
                if literals:
                    clauses.append(literals)
            except ValueError:
                continue
    return clauses

# Fonction pour tester un algorithme donné sur un ensemble de clauses
def tester_algorithme(algo_func, clauses):
    global cpt_dpll
    cpt_dpll = 0
    start_time = time.time()
    resultat = algo_func(clauses)
    elapsed_time = time.time() - start_time
    return resultat, elapsed_time, cpt_dpll

# Liste des fichiers à tester
fichiers = [
    "/content/drive/MyDrive/uuf50/uuf50-03.cnf",
    "/content/drive/MyDrive/uuf50/uuf50-04.cnf",
    "/content/drive/MyDrive/uuf150-01.cnf"
]

# Exécuter les tests
resultats = []
for fichier in fichiers:
    clauses = lire_cnf(fichier)
    dpll_result, dpll_time, dpll_calls = tester_algorithme(DPLL_optimise, clauses)
    resultats.append({
        "Fichier": fichier,
        "DPLL Résultat": dpll_result,
        "DPLL Temps (s)": dpll_time,
        "DPLL Appels": dpll_calls
    })

# Afficher les résultats sous forme de tableau
df_resultats = pd.DataFrame(resultats)
print(df_resultats)


                                     Fichier  DPLL Résultat  DPLL Temps (s)  \
0  /content/drive/MyDrive/uuf50/uuf50-03.cnf          False        0.022535   
1  /content/drive/MyDrive/uuf50/uuf50-04.cnf          False        0.047843   
2       /content/drive/MyDrive/uuf150-01.cnf          False      131.154044   

   DPLL Appels  
0           79  
1          167  
2        84635  


# ***Comparaison des algorithmes***

In [ ]:
import time
import pandas as pd
from collections import Counter

# Fonction pour lire un fichier DIMACS
def lire_cnf(fichier):
    """Lit un fichier CNF au format DIMACS et retourne une liste de clauses."""
    clauses = []
    with open(fichier, 'r') as f:
        for ligne in f:
            ligne = ligne.strip()
            if not ligne or ligne.startswith(('c', 'p', '%', '0')):
                continue
            try:
                literals = list(map(int, ligne.split()))
                if literals and literals[-1] == 0:
                    literals.pop()
                if literals:
                    clauses.append(literals)
            except ValueError:
                continue
    return clauses

# Fonction pour tester un algorithme donné
def tester_algorithme(algo_func, clauses, algo_name):
    """Teste un algorithme sur un ensemble de clauses et mesure les performances."""
    global cpt, cpt_dpll
    cpt = 0
    cpt_dpll = 0
    start_time = time.time()
    resultat = algo_func(clauses)
    elapsed_time = time.time() - start_time
    appels = cpt if algo_name == "DP_optimise" else cpt_dpll
    return resultat, elapsed_time, appels

# Fichiers DIMACS à tester
fichiers = [
    "/content/drive/MyDrive/uf50/uf50-01.cnf",
    "/content/drive/MyDrive/uuf50/uuf50-04.cnf",
    "/content/drive/MyDrive/uuf150-01.cnf"
]

# Comparaison entre DP_optimise et DPLL_optimise
resultats = []

for fichier in fichiers:
    clauses = lire_cnf(fichier)

    # Tester DP_optimise
    dp_result, dp_time, dp_calls = tester_algorithme(DP_optimise, clauses, "DP_optimise")

    # Tester DPLL_optimise
    dpll_result, dpll_time, dpll_calls = tester_algorithme(DPLL_optimise, clauses, "DPLL_optimise")

    # Collecter les résultats
    resultats.append({
        "Fichier": fichier,
        "DP Résultat": dp_result,
        "DP Temps (s)": dp_time,
        "DP Appels": dp_calls,
        "DPLL Résultat": dpll_result,
        "DPLL Temps (s)": dpll_time,
        "DPLL Appels": dpll_calls
    })

# Afficher les résultats sous forme de tableau
df_resultats = pd.DataFrame(resultats)
print(df_resultats)



                                     Fichier  DP Résultat  DP Temps (s)  \
0    /content/drive/MyDrive/uf50/uf50-01.cnf         True      0.011145   
1  /content/drive/MyDrive/uuf50/uuf50-04.cnf        False      0.048775   
2       /content/drive/MyDrive/uuf150-01.cnf        False    140.342233   

   DP Appels  DPLL Résultat  DPLL Temps (s)  DPLL Appels  
0         48           True        0.007969           48  
1        167          False        0.046200          167  
2      84635          False      125.235742        84635  
